<a href="https://colab.research.google.com/github/Cat-sandwich/Labochka_5/blob/main/Labochka_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import main
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

# Пункт 1

Необходимо загрузить исходный набор данных и соответствующие метки классов.

In [4]:
data = pd.read_csv('dataframe.csv')
data.head()

,class_mark,text_review
0,good,Зеленая миля\n'Они помогли ему убить себя и та...
1,good,"Зеленая миля\n'Боже, иногда Зеленая миля так д..."
2,good,Зеленая миля\nОни помогли ему убить себя\n\n«О...
3,good,Зеленая миля\n«Когда призовет Господь на свой ...
4,good,Зеленая миля\nОбвиненный в страшном преступлен...


In [5]:
remove_non_alphabets =lambda x: re.sub(r'[^а-яА-Я]',' ',x)

In [6]:
tokenize = lambda x: word_tokenize(x, language="russian")

In [10]:
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

In [11]:
lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [12]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
print('Processing : [=', end='')
data['text_review'] = data['text_review'].apply(remove_non_alphabets)
print('=', end='')
data['text_review'] = data['text_review'].apply(tokenize) # [ word_tokenize(row) for row in data['email']]
print('=', end='')
data['text_review'] = data['text_review'].apply(stem)
print('=', end='')
data['text_review'] = data['text_review'].apply(leammtizer)
print('=', end='')
data['text_review'] = data['text_review'].apply(lambda x: ' '.join(x))
print('] : Completed', end='')
data.head()


Processing : [=====] : Completed

,class_mark,text_review
0,good,зеленая миля они помогли ему убить себя и так ...
1,good,зеленая миля боже иногда зеленая миля так длин...
2,good,зеленая миля они помогли ему убить себя они по...
3,good,зеленая миля когда призовет господь на свой су...
4,good,зеленая миля обвиненный в страшном преступлени...


In [113]:
max_words = 10000
cv = CountVectorizer(max_features=max_words, stop_words='english')
sparse_matrix = cv.fit_transform(data['text_review']).toarray()

In [114]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(data['label']))

KeyError: ignored